# Hamming weight and Pearson Correlation

We generate a random secret floating number.
Then we use hamming weight model to guess the number

In [1]:
import numpy as np
import pandas as pd
import random

#
# initalize parameter and generate a random number (secret number)
guess_range = (-100, 100.0)
secret_number = random.uniform(guess_range[0], guess_range[1])
prescision = 1e-3

In [2]:
import struct

def float_to_bin(num):
    bits, = struct.unpack('!I', struct.pack('!f', num))
    return "{:032b}".format(bits)

def hamming_weight(num):
    return float_to_bin(num).count('1')

In [3]:
number_tests = 10
known_inputs = np.random.uniform(-1e0, 1e0, 10000)

## Guessing...

In [4]:
while (guess_range[1] - guess_range[0]) > prescision:
    print('*** guess_range =', guess_range)
    range_middle_value = (guess_range[1] + guess_range[0]) / 2.0
    sub_ranges = [(guess_range[0], range_middle_value), (range_middle_value, guess_range[1])]
    low, high = sub_ranges[0]
    guess_val = np.arange(low, high, (high-low)/number_tests)
    hw_low = pd.DataFrame(columns=guess_val, index=range(len(known_inputs)))
    for x in guess_val:
        hw_low[x] = [hamming_weight(x*y) for y in known_inputs]
    hw_low['actual'] = [hamming_weight(secret_number*y) for y in known_inputs]

    low, high = sub_ranges[1]
    guess_val = np.arange(low, high, (high-low)/number_tests)
    hw_high = pd.DataFrame(columns=guess_val, index=range(len(known_inputs)))
    for x in guess_val:
        hw_high[x] = [hamming_weight(x*y) for y in known_inputs]
    hw_high['actual'] = [hamming_weight(secret_number*y) for y in known_inputs]

    low_corr = hw_low.corr(method='pearson')
    high_corr = hw_high.corr(method='pearson')
    
    a = low_corr['actual'].sum()
    b = high_corr['actual'].sum()
    print('low_corr =', a,', high_corr =', b)
    if a > b:
        guess_range = sub_ranges[0]
    else:
        guess_range = sub_ranges[1]

*** guess_range = (-100, 100.0)
low_corr = 1.9021329271394112 , high_corr = 0.9603314464064184
*** guess_range = (-100, 0.0)
low_corr = 2.1595917395892137 , high_corr = 1.6908259798867893
*** guess_range = (-100, -50.0)
low_corr = 1.810282059406428 , high_corr = 2.456719027167893
*** guess_range = (-75.0, -50.0)
low_corr = 2.758483829016159 , high_corr = 2.0993753407504476
*** guess_range = (-75.0, -62.5)
low_corr = 2.92559782841162 , high_corr = 2.573212178444251
*** guess_range = (-75.0, -68.75)
low_corr = 2.48999722234911 , high_corr = 3.4532223876892365
*** guess_range = (-71.875, -68.75)
low_corr = 3.115636601510544 , high_corr = 3.8700982799340133
*** guess_range = (-70.3125, -68.75)
low_corr = 4.243841533401206 , high_corr = 3.477175095302977
*** guess_range = (-70.3125, -69.53125)
low_corr = 3.862470341236342 , high_corr = 4.650765614724012
*** guess_range = (-69.921875, -69.53125)
low_corr = 5.183581473112667 , high_corr = 4.227403635611537
*** guess_range = (-69.921875, -69.7

## Result

In [5]:
print('the secret number =', secret_number)
guessed_number = (guess_range[1] + guess_range[0]) / 2.0
print('the guessed number =', guessed_number)
error_rate = abs((guessed_number - secret_number) / secret_number) * 100
print('error rate = %0.4f' % error_rate, '%')

the secret number = -69.82314691217954
the guessed number = -69.82307434082031
error rate = 0.0001 %
